# AI-Driven Data Intelligence for Smarter Cookie Franchises

Welcome to this demo, where you’ll build a powerful AI agent tailored for a cookie franchise business. This agent is designed to empower franchise owners to analyze customer data, create targeted marketing campaigns, and develop data-driven sales strategies that improve their operations.

By using data intelligence rather than generic insights, franchises can understand top-selling products and create campaigns based on actual sales data.

This notebook will guide you through creating and registering simple functions in Unity Catalog, providing governed access to insights. You'll then build a chat-based AI using these functions, enabling franchises to develop smarter, data-driven campaigns.

Here's what we'll cover:

- Creating and registering SQL functions in Unity Catalog
- Using Langchain to integrate these functions as tools
- Building an AI agent to execute these tools and tackle complex questions

In [0]:
%pip install -q langchain-community==0.2.16 langchain-openai==0.1.19 mlflow==2.15.1 databricks-agents==0.5.0 langchain==0.2.16
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



## Step 1: Function/Tool Creation in Unity Catalog

You'll create two functions:
- A simple query to retrieve an ID based on a city name
- An aggregate query that returns sales data for a given ID

In [0]:
%sql
-- First lets make sure it doesnt already exist
DROP FUNCTION IF EXISTS workspace.default.franchise_by_city;
-- Now we create our first function. This takes in a city name and returns a table of any franchises that are in that city.
-- Note that we've added a comment to the input parameter to help guide the agent later on.
CREATE OR REPLACE FUNCTION
workspace.default.franchise_by_city(
  city_name STRING COMMENT 'City to be searched'
)
returns table(franchiseID BIGINT, name STRING, size STRING)
return
(SELECT franchiseID, name, size from samples.bakehouse.sales_franchises where city=city_name 
     order by size desc)


In [0]:
%sql
-- Test the function we just created
SELECT * from workspace.default.franchise_by_city('Seattle')

franchiseID,name,size
3000038,Dough Delights,XXL
3000014,Sweet Temptations,L


In [0]:
%sql
-- Again check that it exists
DROP FUNCTION IF EXISTS workspace.default.franchise_sales;
-- This function takes an ID as input, and this time does an aggregate to return the sales for that franchise_id.
CREATE OR REPLACE FUNCTION
workspace.default.franchise_sales (
  franchise_id BIGINT COMMENT 'ID of the franchise to be searched'
)
returns table(total_sales BIGINT, total_quantity BIGINT, product STRING)
return
(SELECT SUM(totalPrice) AS total_sales, SUM(quantity) AS total_quantity, product 
FROM samples.bakehouse.sales_transactions 
WHERE franchiseID = franchise_id GROUP BY product)

In [0]:
%sql
-- Check the sales function works - we're going to use the franchise_id from the previous query
Select * from workspace.default.franchise_sales(3000038)

total_sales,total_quantity,product
138,46,Pearly Pies
348,116,Outback Oatmeal
108,36,Austin Almond Biscotti
108,36,Orchard Oasis
51,17,Tokyo Tidbits
189,63,Golden Gate Ginger


In [0]:
import mlflow
import langchain

# This is going to allow us to understand what happens during every part of the agent's execution
mlflow.langchain.autolog(disable=False)

2025/05/11 13:55:48 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of langchain. If you encounter errors during autologging, try upgrading / downgrading langchain to a supported version, or try upgrading MLflow.


## Step 2: Create Agent

In this step, we're going to define three cruicial parts of our agent:
- Tools for the Agent to use
- LLM to serve as the agent's "brains"
- System prompt that defines guidelines for the agent's tasks

In [0]:
from langchain_community.tools.databricks import UCFunctionToolkit
from databricks.sdk import WorkspaceClient
import pandas as pd
import time

w = WorkspaceClient()

def get_shared_warehouse():
    w = WorkspaceClient()
    warehouses = w.warehouses.list()
    for wh in warehouses:
        if wh.name == "Serverless Starter Warehouse":
            if wh.num_clusters == 0:
                w.warehouses.start(wh.id)
                time.sleep(5)
                return wh
            else:
                return wh 
    raise Exception("Couldn't find any Warehouse to use. Please start the serverless SQL Warehouse for this code to run.")
    
wh_id = get_shared_warehouse().id

def get_tools():
    return (
        UCFunctionToolkit(warehouse_id=wh_id)
        .include("workspace.default.*")
        .get_tools())

In [0]:
from langchain_community.chat_models.databricks import ChatDatabricks

# We're going to use llama 3.1 because it's tool enabled and works great. Keep temp at 0 to make it more deterministic.
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.0,
    streaming=False)

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks

# This defines our agent's system prompt. Here we can tell it what we expect it to do and guide it on using specific functions. 

def get_prompt(history = [], prompt = None):
    if not prompt:
            prompt = """You are a helpful assistant for a global company that oversees cookie stores. Your task is to help store owners understand more about their products and sales metrics. You have the ability to execute functions as follows: 

            Use the franchise_by_city function to retrieve the franchiseID for a given city name.

            Use the franchise_sales function to retrieve the cookie sales for a given franchiseID. This should be run for each franchiseID. Do not ask the user if they want to see another store, just run it for ALL franchiseIDs.

    Make sure to call the function for each step and provide a coherent final response to the user. Don't mention tools to your users. Don't skip to the next step without ensuring the function was called and a result was retrieved. Only answer what the user is asking for."""
    return ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("human", "{messages}"),
            ("placeholder", "{agent_scratchpad}"),
    ])

In [0]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, Tool, load_tools

prompt = get_prompt()
tools = get_tools()
agent = create_openai_tools_agent(llm, tools, prompt)

# Here we're collecting the defined pieces and putting them together to create our Agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [0]:
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Very basic chain that allows us to pass the input (messages) into the Agent and collect the (output) as a string
agent_str = ({ "messages": itemgetter("messages")} | agent_executor | itemgetter("output") | StrOutputParser() )

In [0]:
# Lets ask our Compound AI Agent to generate an Instagram post. This requires it to:
#     1. Look up what stores are in Seattle
#     2. Use sales data to look up the best selling cookie at that store

answer=agent_str.invoke({"messages": "What is the best selling cookie in our Seattle stores?"})



> Entering new AgentExecutor chain...

Invoking: `workspace__default__franchise_by_city` with `{'city_name': 'Seattle'}`


{"format": "CSV", "value": "franchiseID,name,size\n3000038,Dough Delights,XXL\n3000014,Sweet Temptations,L\n", "truncated": false}
Invoking: `workspace__default__franchise_sales` with `{'franchise_id': 3000038}`


{"format": "CSV", "value": "total_sales,total_quantity,product\n108,36,Austin Almond Biscotti\n348,116,Outback Oatmeal\n138,46,Pearly Pies\n189,63,Golden Gate Ginger\n51,17,Tokyo Tidbits\n108,36,Orchard Oasis\n", "truncated": false}
Invoking: `workspace__default__franchise_sales` with `{'franchise_id': 3000014}`


{"format": "CSV", "value": "total_sales,total_quantity,product\n27,9,Austin Almond Biscotti\n48,16,Outback Oatmeal\n42,14,Pearly Pies\n42,14,Golden Gate Ginger\n36,12,Tokyo Tidbits\n30,10,Orchard Oasis\n", "truncated": false}The best selling cookie in the Seattle stores is the Outback Oatmeal cookie from the Dough Delights store, with 116 unit

Trace(request_id=tr-549a1d3dbbdd41229598dcb885ba9e5d)

## Excited for more?

- Catch the full demo in action [here](https://www.youtube.com/watch?v=UfbyzK488Hk&t=3501s). 
- Take the next step and build a [RAG-based chatbot](https://www.databricks.com/resources/demos/tutorials/data-science-and-ai/lakehouse-ai-deploy-your-llm-chatbot?itm_data=demo_center) with added contextual depth!